In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import glob
import re

# Import Data

In [2]:
name = "CloudResearch-1_24.08.12"
df_full = pd.read_csv("/work/Data/" + name + ".csv")[2:]
df_full

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,H Attributional Conf_5,Housing,Living With,Area Type,Feedback,Comments,Longitudinal,participantId,assignmentId,projectId
2,2024-06-10 16:25:22,2024-06-10 16:36:39,IP Address,100,677,True,2024-06-10 16:36:40,R_3Tq3KIlfVrBymDk,anonymous,EN,...,70,Live in parent’s house or apartment,"Parent(s),Sibling(s)",Suburban,NaN,NaN,Yes,137C0B796EF34E5B9E2CAE98747DCFCD,955f9512-6dd3-4ee5-b7a3-306dab16d361,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
3,2024-06-10 16:22:15,2024-06-10 16:37:20,IP Address,100,905,True,2024-06-10 16:37:21,R_77JLXrlZNFvFjZA,anonymous,EN,...,83,Live in parent’s house or apartment,Parent(s),Suburban,NaN,NaN,Yes,5A606E625C384C9AB48B2DE67C6DEED5,3b63811c-ce61-4052-a257-70d3b9cd981c,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
4,2024-06-10 16:26:47,2024-06-10 16:39:38,IP Address,100,770,True,2024-06-10 16:39:38,R_1xOvnrl7kD6ysKd,anonymous,EN,...,95,Live in own house or apartment,Unmarried partner,Urban,NaN,NaN,Yes,9505A6052BF848ABAB7E7E65C2BE3B0C,84757a77-0fde-413c-8b90-5e426be15a3d,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
5,2024-06-10 16:22:16,2024-06-10 16:39:57,IP Address,100,1060,True,2024-06-10 16:39:57,R_6i2xfeJezDf9zMB,anonymous,EN,...,68,Live in own house or apartment,Living alone,Urban,NaN,NaN,Yes,61FFB453863242FDA4ED125D8E818F3C,9f915c98-c2a1-4fdd-9860-6f0991ad03c6,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
6,2024-06-10 16:25:01,2024-06-10 16:40:11,IP Address,100,909,True,2024-06-10 16:40:11,R_2oYYZFDVjHmNVrC,anonymous,EN,...,80,Live in own house or apartment,Unmarried partner,Rural,none,none,Yes,DA20201B90AC4D22A238BF40FF470DEC,740be7f7-17a0-4980-a03d-f802765a5293,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,2024-08-02 05:23:31,2024-08-02 05:30:58,IP Address,38,447,False,2024-08-09 05:30:59,R_2CjNdcv5vSYmv7m,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A57B7C7EF3B741F6ABDB8F3638368DE0,8327f22e-f900-47e9-aa4c-4e0dabc691ea,514e18cc-7279-4b9f-b363-11fbe22f3e1a
698,2024-08-09 05:43:55,2024-08-09 06:00:07,IP Address,100,972,True,2024-08-09 06:00:08,R_1UHBs8uY4WYlH0d,anonymous,EN,...,91,Live in own house or apartment,Spouse,Suburban,NaN,NaN,Yes,976C3FF9847D451D8DCC9CDEB0218DCD,7d86b4a5-d37f-4ff2-8a75-bd01a0079873,fb139a63-e822-4967-af9c-bb53107a5276
699,2024-08-09 18:43:09,2024-08-09 19:01:59,IP Address,100,1129,True,2024-08-09 19:01:59,R_196t5uDWJDsgjuF,anonymous,EN,...,85,Live in own house or apartment,Other nonrelative,Urban,Nothing unclear!,Thanks for letting me participate!,Yes,A658EFC92E94458DAA116A5D63366360,31e55088-52f5-46e6-8689-c1202ce535af,fb139a63-e822-4967-af9c-bb53107a5276
700,2024-08-09 20:41:09,2024-08-09 20:51:33,IP Address,100,624,True,2024-08-09 20:51:34,R_6rp9rQAQeELF72t,anonymous,EN,...,65,Live in parent’s house or apartment,Parent(s),Suburban,NaN,NaN,Yes,F29184B2093B49E7A7FAEA972C2241AD,ea4db047-f0cd-4d85-9f27-523a775cce72,514e18cc-7279-4b9f-b363-11fbe22f3e1a


In [3]:
df_full.loc[(df_full['Longitudinal'] == 'No')]['participantId']

36     DE5E843B32A0413BB3E23F052FC2A686
301    7D598C21E4DB4766897D1ED72BC6D35B
638    4E4DC1169B3845628FD80D782B89FF07
661    9F2E2B4B99BE486C925A669DD04617E7
Name: participantId, dtype: object

Filter for users who have finished the survey. 

In [4]:
#Filter non-user out
# df = df_full.loc[(df_full['Usage'] == 'Yes') & (df_full['Finished'] == 'True')].copy().reset_index()
df = df_full.loc[df_full['Finished'] == 'True'].copy().reset_index()
df

,index,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,...,H Attributional Conf_5,Housing,Living With,Area Type,Feedback,Comments,Longitudinal,participantId,assignmentId,projectId
0,2,2024-06-10 16:25:22,2024-06-10 16:36:39,IP Address,100,677,True,2024-06-10 16:36:40,R_3Tq3KIlfVrBymDk,anonymous,...,70,Live in parent’s house or apartment,"Parent(s),Sibling(s)",Suburban,NaN,NaN,Yes,137C0B796EF34E5B9E2CAE98747DCFCD,955f9512-6dd3-4ee5-b7a3-306dab16d361,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
1,3,2024-06-10 16:22:15,2024-06-10 16:37:20,IP Address,100,905,True,2024-06-10 16:37:21,R_77JLXrlZNFvFjZA,anonymous,...,83,Live in parent’s house or apartment,Parent(s),Suburban,NaN,NaN,Yes,5A606E625C384C9AB48B2DE67C6DEED5,3b63811c-ce61-4052-a257-70d3b9cd981c,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
2,4,2024-06-10 16:26:47,2024-06-10 16:39:38,IP Address,100,770,True,2024-06-10 16:39:38,R_1xOvnrl7kD6ysKd,anonymous,...,95,Live in own house or apartment,Unmarried partner,Urban,NaN,NaN,Yes,9505A6052BF848ABAB7E7E65C2BE3B0C,84757a77-0fde-413c-8b90-5e426be15a3d,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
3,5,2024-06-10 16:22:16,2024-06-10 16:39:57,IP Address,100,1060,True,2024-06-10 16:39:57,R_6i2xfeJezDf9zMB,anonymous,...,68,Live in own house or apartment,Living alone,Urban,NaN,NaN,Yes,61FFB453863242FDA4ED125D8E818F3C,9f915c98-c2a1-4fdd-9860-6f0991ad03c6,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
4,6,2024-06-10 16:25:01,2024-06-10 16:40:11,IP Address,100,909,True,2024-06-10 16:40:11,R_2oYYZFDVjHmNVrC,anonymous,...,80,Live in own house or apartment,Unmarried partner,Rural,none,none,Yes,DA20201B90AC4D22A238BF40FF470DEC,740be7f7-17a0-4980-a03d-f802765a5293,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,693,2024-08-08 17:30:42,2024-08-08 17:46:22,IP Address,100,939,True,2024-08-08 17:46:23,R_40JjHfQYavKjN2N,anonymous,...,90,Live in parent’s house or apartment,Parent(s),Suburban,NaN,NaN,Yes,03121AB929C94892A0B1A118B67FE832,46f07762-50df-4ea1-ba97-558dc244b0fa,fb139a63-e822-4967-af9c-bb53107a5276
670,698,2024-08-09 05:43:55,2024-08-09 06:00:07,IP Address,100,972,True,2024-08-09 06:00:08,R_1UHBs8uY4WYlH0d,anonymous,...,91,Live in own house or apartment,Spouse,Suburban,NaN,NaN,Yes,976C3FF9847D451D8DCC9CDEB0218DCD,7d86b4a5-d37f-4ff2-8a75-bd01a0079873,fb139a63-e822-4967-af9c-bb53107a5276
671,699,2024-08-09 18:43:09,2024-08-09 19:01:59,IP Address,100,1129,True,2024-08-09 19:01:59,R_196t5uDWJDsgjuF,anonymous,...,85,Live in own house or apartment,Other nonrelative,Urban,Nothing unclear!,Thanks for letting me participate!,Yes,A658EFC92E94458DAA116A5D63366360,31e55088-52f5-46e6-8689-c1202ce535af,fb139a63-e822-4967-af9c-bb53107a5276
672,700,2024-08-09 20:41:09,2024-08-09 20:51:33,IP Address,100,624,True,2024-08-09 20:51:34,R_6rp9rQAQeELF72t,anonymous,...,65,Live in parent’s house or apartment,Parent(s),Suburban,NaN,NaN,Yes,F29184B2093B49E7A7FAEA972C2241AD,ea4db047-f0cd-4d85-9f27-523a775cce72,514e18cc-7279-4b9f-b363-11fbe22f3e1a


In [5]:
df_full.loc[df_full['Finished'] != 'True']

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,H Attributional Conf_5,Housing,Living With,Area Type,Feedback,Comments,Longitudinal,participantId,assignmentId,projectId
219,2024-06-10 16:30:57,2024-06-10 16:32:36,IP Address,15,99,False,2024-06-17 16:32:41,R_5q0nGiUz9CU1qMh,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,034AA152656B4D7A978ECD958E2EC126,587fe116-43ef-4448-ab1d-120132743c57,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
220,2024-06-10 17:09:47,2024-06-10 17:17:18,IP Address,15,451,False,2024-06-17 17:17:22,R_70ufVIvJP9gcTmi,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DF5E6DF781444634BFCFB953677B38F2,3e7fb86e-8925-4b27-bdfc-4a21a6487df0,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
221,2024-06-10 23:34:52,2024-06-10 23:39:51,IP Address,15,299,False,2024-06-17 23:39:53,R_8hyE800zTZLz4NV,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5D083AD0751E47248EAA013D74068D47,2ab363e6-1c34-4e4a-bbc6-e59a2062b1f6,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
222,2024-06-10 18:02:43,2024-06-11 09:04:37,IP Address,49,54114,False,2024-06-18 09:04:40,R_3tHD0qadPNfDtWV,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9A5B731873534EC5ADF37FC60ED9842E,06ee4ed1-71c1-42b6-a657-77ba15a3e6c5,610a1c01-335b-4ddb-8e7a-a4b4b4b57675
223,2024-06-11 20:40:14,2024-06-11 20:42:40,IP Address,34,146,False,2024-06-18 20:42:44,R_66hg04ueK9W1n4F,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75B7E17B71B840568DF71AE43662AD2D,2b58ff54-b435-4a07-b406-5998146efe6d,54f50144-3e27-4d16-b58f-3aa07fdfb059
224,2024-06-11 21:05:48,2024-06-11 21:07:56,IP Address,15,128,False,2024-06-18 21:07:57,R_15Totku956Lcp2T,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,077BD6F383AE49B08729AE107FD53166,b76853c3-5506-4166-af34-6df5b1c1040c,54f50144-3e27-4d16-b58f-3aa07fdfb059
292,2024-06-12 16:18:58,2024-06-12 16:46:03,IP Address,26,1624,False,2024-06-19 16:46:06,R_6niW9wVZUCpIXJD,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EC1B00EB2D7E4005B099951B93D92E1F,c1ad3db9-42f6-48cb-a25d-9adf90916ecc,60964d4b-2410-446e-9cb3-38a550481a42
304,2024-06-19 11:14:45,2024-06-19 12:19:27,IP Address,34,3882,False,2024-06-26 12:19:29,R_1egmBASBr1WfiIB,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,972F5FC312B54088BD1700B63688870E,12cc856c-e020-4072-b936-b1efcd57ebd8,148a3896-3b51-4827-a70f-abe8ef903019
679,2024-08-01 08:04:39,2024-08-01 08:08:06,IP Address,7,207,False,2024-08-08 08:08:10,R_7abQxsCt69eu5IU,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001A08BD6CD14051966DA95937FA3CAB,d1b32936-232f-4ef3-bc90-16a8e16dd504,292efbac-c818-4a67-975c-5809d9194c64
680,2024-08-01 08:33:58,2024-08-01 08:34:58,IP Address,15,59,False,2024-08-08 08:35:01,R_6tAaZmQlKNjvZde,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22FD8F3C32A740E3A53BDFDCEFC06715,5e045ad4-ba76-4490-a633-aceddae3195d,fb139a63-e822-4967-af9c-bb53107a5276


# Calculate ULS Score

In [6]:
uls_s = df.columns.get_loc("ULS1")
uls_e = uls_s + 8

uls = df.iloc[:, uls_s:uls_e]

score_dict = {"Never" : 1, 
                "Rarely" : 2,
                "Sometimes" : 3,
                "Often" : 4} 

score_dict_rev = {"Never" : 4, 
                "Rarely" : 3,
                "Sometimes" : 2,
                "Often" : 1} 

uls_norm = uls.iloc[:,0:5]
uls_norm2 = uls.iloc[:,6]
uls_norm = uls_norm.replace(score_dict)  
uls_norm2 = uls_norm2.replace(score_dict)  
uls_rev =  uls.iloc[:,5]
uls_rev = uls_rev.replace(score_dict_rev)
uls_rev2 =  uls.iloc[:,7]
uls_rev2 = uls_rev2.replace(score_dict_rev)
uls_scores = pd.concat([uls_norm, uls_norm2, uls_rev, uls_rev2], axis=1)
uls_scores

,ULS1,ULS2,ULS3,ULS4,ULS5,ULS7,ULS6,ULS8
0,3,1,2,3,3,1,1,2
1,4,4,4,4,4,4,3,3
2,1,1,1,1,1,2,1,1
3,3,3,3,3,3,3,2,3
4,1,1,2,1,1,1,3,2
...,...,...,...,...,...,...,...,...
669,2,1,2,2,1,1,1,1
670,2,1,2,2,1,2,2,2
671,3,2,3,3,3,2,2,3
672,3,3,4,4,2,3,4,4


In [7]:
df['ULS-8'] = uls_scores.sum(axis=1)
df['ULS-8']

0      16
1      30
2       9
3      23
4      12
       ..
669    11
670    14
671    21
672    27
673    19
Name: ULS-8, Length: 674, dtype: int64

# Convert Usage to Numerical Score

In [8]:
def mcToNum(responses):
    # Split the responses by comma
    response_list = responses.split(',')
    
    # Initialize a list to store numerical values
    numerical_values = []

    # Convert to 0-30 linear scale 
    response_mapping = {"Never" : 0,
            "Daily" : 30, 
            "A few times a week" : 24,
            "Around once a week" : 18,
            "A few times a month" : 12,
            "Once a month or less" : 6, 
            "<15 min" : 5, 
            "15-30 min" : 10,
            "30 min - 1h" : 15,
            "1h - 2h" : 20,
            "2h - 4h" : 25,
            "4h+" : 30} 
    
    # Collect values for multiple responses
    for response in response_list:
        if response in response_mapping.keys():
            numerical_values.append(response_mapping.get(response, 0))

    # Calculate the average value
    if len(numerical_values) > 0: 
        # max_value = max(numerical_values)
        average_value = sum(numerical_values) / len(numerical_values)
    else: 
        average_value = 0
    
    #print("***")
    #print(response_list)
    #print(len(numerical_values))
    #print(average_value)

    return average_value

freq = df['Chatbot Frequency'].fillna("Never")
slen = df['Session Length'].fillna("Never")

freq_num = freq.apply(mcToNum)
slen_num = slen.apply(mcToNum)
df['Frequency Score'] = (freq_num) / 30 * 100
df['Session Length Score'] = (slen_num) / 30 * 100

In [9]:
df['Session Length'].value_counts()

15-30 min                                            157
<15 min                                              135
30 min - 1h                                           45
<15 min,15-30 min                                     23
1h - 2h                                               13
15-30 min,30 min - 1h                                 10
<15 min,15-30 min,30 min - 1h                          5
30 min - 1h,1h - 2h                                    5
4h+                                                    3
15-30 min,30 min - 1h,1h - 2h                          2
<15 min,15-30 min,30 min - 1h,1h - 2h,2h - 4h,4h+      1
<15 min,15-30 min,30 min - 1h,1h - 2h                  1
30 min - 1h,1h - 2h,2h - 4h,4h+                        1
2h - 4h                                                1
<15 min,30 min - 1h                                    1
15-30 min,4h+                                          1
Name: Session Length, dtype: int64

In [10]:
# Define a mapping dictionary for Session Length Score to session length categories
session_length_mapping = {
    16.67: "<15 min",
    33.33: "15-30 min",
    50.00: "30 min - 1h",
    66.67: "1h - 2h",
    83.33: "2h - 4h",
    100.00: "4h+",

}

# Round the 'Session Length Score' values to 2 decimal places before mapping
df['updated_session_length'] = df['Session Length Score'].round(2).map(session_length_mapping)

# Check the result
print(df[['Session Length Score', 'updated_session_length']].value_counts())

Session Length Score  updated_session_length
33.333333             15-30 min                 163
16.666667             <15 min                   135
50.000000             30 min - 1h                47
66.666667             1h - 2h                    14
100.000000            4h+                         3
83.333333             2h - 4h                     1
dtype: int64


In [11]:
print(df['Session Length Score'].value_counts())

0.000000      270
33.333333     163
16.666667     135
50.000000      47
25.000000      23
66.666667      14
41.666667      11
58.333333       6
100.000000      3
75.000000       1
83.333333       1
Name: Session Length Score, dtype: int64


# Calculate Scales

In [12]:
def calcScale(df, startCol, name, length, mn=1, mx=7, revList=[], calcType="avg", drop=True): 
    i_s = df.columns.get_loc(startCol)
    i_e = i_s + length 

    sub_df = df.iloc[:, i_s:i_e].astype(float)

    for revItem in revList: 
        sub_df.iloc[:, revItem-1] = mx + mn - sub_df.iloc[:, revItem-1]

    if calcType == "add": 
        df[name] = sub_df.sum(axis=1)
    elif calcType == "avg": 
        df[name] = sub_df.mean(axis=1)
    
    # Set result to NaN if all values in the row are NaN
    df[name] = df[name].where(~sub_df.isna().all(axis=1), np.nan)
    
    # Drop the used columns
    if drop:
        df.drop(df.columns[i_s:i_e], axis=1, inplace=True)

    return df

In [13]:
# Issue with GPIUS column names - clean column names in copy before calculating
df2 = df.copy(deep=True)

# Function to clean column names
def clean_column_names(df):
    df.columns = [re.sub(r'[^a-zA-Z0-9]', '_', col) for col in df.columns]
    return df

# Clean the column names
df2 = clean_column_names(df2)

i_s = df2.columns.get_loc("Adapted_GPIUS_2_1")
i_e = i_s + 6 

sub_df = df2.iloc[:, i_s:i_e].astype(float)

df['GPIUS2'] = sub_df.sum(axis=1)
df['GPIUS2'] = df['GPIUS2'].where(~sub_df.isna().all(axis=1), np.nan)
df.drop(df.columns[i_s:i_e], axis=1, inplace=True)

In [14]:
calcScale(df, "SUS_1", name="SUS Avg", length=8, revList=[3, 5, 8], calcType="avg")
calcScale(df, "AI Attitudes_1", name="AI Attitudes Score", length=8, calcType="avg")

calcScale(df, "State Empathy - C_1", name="C State Empathy", length=12, mx=5, drop=False)
calcScale(df, "State Empathy - C_1", name="C SEmp Affective", length=4, mx=5, calcType="avg")
calcScale(df, "State Empathy - C_5", name="C SEmp Cognitive", length=4, mx=5, calcType="avg")
calcScale(df, "State Empathy - C_9", name="C SEmp Associative", length=4, mx=5, calcType="avg")

calcScale(df, "Chatbot Attract_1", name="C Social Attract", revList=[3,4], length=4)
calcScale(df, "Chatbot Attract_5", name="C Phys Attract", revList=[4], length=4)
calcScale(df, "Chatbot Attract_9", name="C Task Attract", revList=[1,2], length=4)

calcScale(df, "Homophily - Chatbots_1", name="C Homophily", revList=[5], length=6)
calcScale(df, "Attributional Conf._1", name="C Attrib Conf", length=4)

calcScale(df, "HCI/HHI_1", name="HCI-HHI Expectation", length=2)
calcScale(df, "HCI/HHI_3", name="HCI-HHI Trust", length=2)
calcScale(df, "HCI/HHI_5", name="HCI-HHI Substitution", length=3)

calcScale(df, "MSPSS_1", name="MSPSS Score", length=12)
calcScale(df, "B-RSES_1", name="B-RSES Score", revList=[1,2], length=5)

calcScale(df, "HHI_1", name="HHI Trust - Unfam", length=2)
calcScale(df, "HHI_3", name="HHI CommIntent - Unfam", length=2)
calcScale(df, "HHI_5", name="HHI Trust - Familiar", length=2)
calcScale(df, "HHI_7", name="HHI CommIntent - Familiar", length=2)

calcScale(df, "State Empathy - H_1", name="H State Empathy", length=12, mx=5, drop=False)
calcScale(df, "State Empathy - H_1", name="H SEmp Affective", length=4, mx=5)
calcScale(df, "State Empathy - H_5", name="H SEmp Cognitive", length=4, mx=5)
calcScale(df, "State Empathy - H_9", name="H SEmp Associative", length=4, mx=5)

calcScale(df, "Human Attract_1", name="H Social Attract", revList=[3,4], length=4)
calcScale(df, "Human Attract_5", name="H Task Attract", revList=[1,2], length=4)

calcScale(df, "Homophily_1", name="H Homophily", revList=[5], length=6)
calcScale(df, "H Attributional Conf_1", name="C Attrib Conf", length=4)

,index,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,...,HHI CommIntent - Unfam,HHI Trust - Familiar,HHI CommIntent - Familiar,H State Empathy,H SEmp Affective,H SEmp Cognitive,H SEmp Associative,H Social Attract,H Task Attract,H Homophily
0,2,2024-06-10 16:25:22,2024-06-10 16:36:39,IP Address,100,677,True,2024-06-10 16:36:40,R_3Tq3KIlfVrBymDk,anonymous,...,6.0,7.0,7.0,4.583333,4.00,4.75,5.00,6.75,7.00,4.166667
1,3,2024-06-10 16:22:15,2024-06-10 16:37:20,IP Address,100,905,True,2024-06-10 16:37:21,R_77JLXrlZNFvFjZA,anonymous,...,1.5,4.0,4.0,2.333333,2.25,2.75,2.00,5.25,4.00,4.833333
2,4,2024-06-10 16:26:47,2024-06-10 16:39:38,IP Address,100,770,True,2024-06-10 16:39:38,R_1xOvnrl7kD6ysKd,anonymous,...,4.0,6.5,7.0,4.750000,4.75,4.75,4.75,4.25,4.00,6.666667
3,5,2024-06-10 16:22:16,2024-06-10 16:39:57,IP Address,100,1060,True,2024-06-10 16:39:57,R_6i2xfeJezDf9zMB,anonymous,...,2.5,2.0,2.5,3.583333,3.25,3.75,3.75,4.00,4.25,4.833333
4,6,2024-06-10 16:25:01,2024-06-10 16:40:11,IP Address,100,909,True,2024-06-10 16:40:11,R_2oYYZFDVjHmNVrC,anonymous,...,1.0,6.0,5.5,4.500000,4.25,4.75,4.50,5.50,5.50,5.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,693,2024-08-08 17:30:42,2024-08-08 17:46:22,IP Address,100,939,True,2024-08-08 17:46:23,R_40JjHfQYavKjN2N,anonymous,...,5.0,7.0,7.0,5.000000,5.00,5.00,5.00,7.00,7.00,6.666667
670,698,2024-08-09 05:43:55,2024-08-09 06:00:07,IP Address,100,972,True,2024-08-09 06:00:08,R_1UHBs8uY4WYlH0d,anonymous,...,2.5,7.0,7.0,4.583333,4.25,4.75,4.75,7.00,7.00,6.166667
671,699,2024-08-09 18:43:09,2024-08-09 19:01:59,IP Address,100,1129,True,2024-08-09 19:01:59,R_196t5uDWJDsgjuF,anonymous,...,6.0,6.5,7.0,4.500000,4.00,5.00,4.50,7.00,5.75,5.500000
672,700,2024-08-09 20:41:09,2024-08-09 20:51:33,IP Address,100,624,True,2024-08-09 20:51:34,R_6rp9rQAQeELF72t,anonymous,...,2.0,4.0,4.5,3.750000,3.50,4.25,3.50,6.25,6.25,6.000000


In [15]:
# BFI-10 
i = df.columns.get_loc("BFI-10_1")
rev = 6
df["BFI Extraversion"] = (rev - df.iloc[:, i].astype(float) + df.iloc[:, i+4].astype(float)) * 0.5 
df["BFI Agreeableness"] = (df.iloc[:, i+1].astype(float) + rev - df.iloc[:, i+6].astype(float)) * 0.5 #fixed
df["BFI Conscientious"] = (rev - df.iloc[:, i+2].astype(float) + df.iloc[:, i+7].astype(float)) * 0.5 
df["BFI Neurotic"] = (rev - df.iloc[:, i+3].astype(float) + df.iloc[:, i+8].astype(float)) * 0.5 #fixed
df["BFI Openness"] = (rev - df.iloc[:, i+4].astype(float) + df.iloc[:, i+9].astype(float)) * 0.5 

In [16]:
# LSNS
i_s = df.columns.get_loc("LSNS1")
i_e = i_s + 6  
df_LSNS = df.iloc[:, i_s:i_e]

LSNS_dict = {"None" : 0, 
            "One" : 1,
            "Two" : 2,
            "Three or four" : 3,
            "Five to eight" : 4,
            "Nine or more" : 5} 


df_LSNS = df_LSNS.replace(LSNS_dict)  
df["LSNS Score"] = df_LSNS.sum(axis=1)

In [17]:
# import the 1d (demographic) csvs as a dataframe
#demographics_files = glob.glob('/work/Data/CloudResearch-1d_*.csv')
#demographics_dfs = [pd.read_csv(file) for file in demographics_files]

# combine the dataframes (concatenate as rows)
#concatenated_demographics = pd.concat(demographics_dfs, ignore_index=True, axis=0)

In [18]:
# import the 1d_current (demographic) csvs as a dataframe
demographics_files_current = glob.glob('/work/Data/CloudResearch-1d_current*.csv')
demographics_dfs_current = [pd.read_csv(file) for file in demographics_files_current]

# combine the dataframes (concatenate as rows)
concatenated_demographics_current = pd.concat(demographics_dfs_current, ignore_index=True, axis=0)
concatenated_demographics_current['Group'] = 'current'

# import the 1d_new (demographic) csvs as a dataframe
demographics_files_new = glob.glob('/work/Data/CloudResearch-1d_new*.csv')
demographics_dfs_new = [pd.read_csv(file) for file in demographics_files_new]

# combine the dataframes (concatenate as rows)
concatenated_demographics_new = pd.concat(demographics_dfs_new, ignore_index=True, axis=0)
concatenated_demographics_new['Group'] = 'new'

# import the 1d_non (demographic) csvs as a dataframe
demographics_files_non = glob.glob('/work/Data/CloudResearch-1d_non*.csv')
demographics_dfs_non = [pd.read_csv(file) for file in demographics_files_non]

# combine the dataframes (concatenate as rows)
concatenated_demographics_non = pd.concat(demographics_dfs_non, ignore_index=True, axis=0)
concatenated_demographics_non['Group'] = 'non'

# concatenate the dataframes
concatenated_demographics = pd.concat([concatenated_demographics_current, concatenated_demographics_new, concatenated_demographics_non], ignore_index=True, axis=0)

In [19]:
# merge with df dataframe by matching participant ID in df and the demographic dataframe
merged_df = df.merge(
    concatenated_demographics,
    left_on='participantId',  # Column name in existing_df
    right_on='ParticipantId',  # Column name in concatenated_demographics
    how='inner'  # This keeps rows that match in both dataframes
)

merged_df['participantId'] = merged_df['participantId'].fillna(merged_df['ParticipantId'])
merged_df = merged_df.drop('ParticipantId', axis=1)
merged_df

,index,StartDate,EndDate,Status_x,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,...,Occupation Field,Relationship/Marital Status,Political Party,Ethnicity,Gender,Country Of Residence,Household Income,Race,Employment Status,Group
0,2,2024-06-10 16:25:22,2024-06-10 16:36:39,IP Address,100,677,True,2024-06-10 16:36:40,R_3Tq3KIlfVrBymDk,anonymous,...,Hospitality & Tourism,Single,Democrat,"No, not of Hispanic, Latino, or Spanish origin",Man,United States,"$150,000-$174,999",Filipino,Full-time,current
1,3,2024-06-10 16:22:15,2024-06-10 16:37:20,IP Address,100,905,True,2024-06-10 16:37:21,R_77JLXrlZNFvFjZA,anonymous,...,Other,Single,Independent,"No, not of Hispanic, Latino, or Spanish origin",Woman,United States,"$50,000-$59,999",White,Part-time,current
2,4,2024-06-10 16:26:47,2024-06-10 16:39:38,IP Address,100,770,True,2024-06-10 16:39:38,R_1xOvnrl7kD6ysKd,anonymous,...,Information Technology,Single,Independent,"No, not of Hispanic, Latino, or Spanish origin",Man,United States,"$100,000-$124,999",White,Full-time,current
3,5,2024-06-10 16:22:16,2024-06-10 16:39:57,IP Address,100,1060,True,2024-06-10 16:39:57,R_6i2xfeJezDf9zMB,anonymous,...,Manufacturing,Single,Republican,"No, not of Hispanic, Latino, or Spanish origin",Man,United States,"$50,000-$59,999",Black or African American,Full-time,current
4,6,2024-06-10 16:25:01,2024-06-10 16:40:11,IP Address,100,909,True,2024-06-10 16:40:11,R_2oYYZFDVjHmNVrC,anonymous,...,Information Technology,In a relationship,Something else,"No, not of Hispanic, Latino, or Spanish origin",Man,United Kingdom,"$20,000-$29,999",White,Full-time,current
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,693,2024-08-08 17:30:42,2024-08-08 17:46:22,IP Address,100,939,True,2024-08-08 17:46:23,R_40JjHfQYavKjN2N,anonymous,...,"Science, Technology, Engineering & Mathematics",Single,Prefer not to say,"No, not of Hispanic, Latino, or Spanish origin",Man,Canada,Prefer not to say,Chinese,Full-time,new
670,698,2024-08-09 05:43:55,2024-08-09 06:00:07,IP Address,100,972,True,2024-08-09 06:00:08,R_1UHBs8uY4WYlH0d,anonymous,...,Information Technology,Married,Democrat,"No, not of Hispanic, Latino, or Spanish origin",Man,United States,"$200,000-$224,999",White,Full-time,new
671,699,2024-08-09 18:43:09,2024-08-09 19:01:59,IP Address,100,1129,True,2024-08-09 19:01:59,R_196t5uDWJDsgjuF,anonymous,...,Other,Single,Democrat,"No, not of Hispanic, Latino, or Spanish origin",Man,United States,"$30,000-$39,999",White,Part-time,new
672,700,2024-08-09 20:41:09,2024-08-09 20:51:33,IP Address,100,624,True,2024-08-09 20:51:34,R_6rp9rQAQeELF72t,anonymous,...,I’d Rather Not Say,Single,Republican,"No, not of Hispanic, Latino, or Spanish origin",Man,United States,"$20,000-$29,999",White,Student,current


In [20]:
merged_df["ULS1"]

0      Sometimes
1          Often
2          Never
3      Sometimes
4          Never
         ...    
669       Rarely
670       Rarely
671    Sometimes
672    Sometimes
673    Sometimes
Name: ULS1, Length: 674, dtype: object

In [21]:
merged_df.to_csv("/work/Processed Data/" + name + "_processed.csv")